In [5]:
# Import du paquet PythonMIP et de toutes ses fonctionnalités
from mip import *
# Import du paquet time pour calculer le temps de résolution
import time 
import InstanceReader2

In [6]:
def solve(data, L, K):
    # Création du modèle vide 
    model = Model(name = "MTZ-AF", sense = mip.MAXIMIZE, solver_name="CBC")

    # Création des variables y, u et t
    y = []
    u = []
    t = []
    for i in range(data.id_max+1):
        y.append([])
        u.append([])
        for j in range(data.id_max+1):
            y[i].append(model.add_var(name="y(" + str(i) + "," + str(j) + ")", var_type=BINARY))
            u[i].append(model.add_var(name="u(" + str(i) + "," + str(j) + ")", var_type=BINARY))
    for k in range(data.id_max+1):
        t.append(model.add_var(name="t(" + str(k)  + ")", lb=0, ub=(L-1) , var_type=CONTINUOUS))
    
    
    ### Fonction objectif
    model += (xsum(data.cost[i][j] * (y[i][j]+u[i][j])  for (i,j) in (data.A) ))
    
    ### Contraintes
    
    # Contrainte 5 et 6
    for i in data.P:
        model += (xsum(u[i][j] for j in data.allPatients[int(data.index[int(i)])]) 
                  - xsum(u[int(data.allDonorsInP[i][j])][i] for j in range(1,int(data.allDonorsInP[i][0])+1) ) == 0)
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])]) 
                  - xsum(y[int(data.allDonors[i][j])][i] for j in range(1,int(data.allDonors[i][0])+1) ) == 0)
    
    # Contrainte 7
    for i in data.P:
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])]) 
                  - xsum(u[i][j] for j in data.allPatients[int(data.index[int(i)])] ) <= 1)
    
    # Contrainte 8
    for i in data.N:
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])]) <= 1)
    
    # Contrainte 9 ##Pas complète
    pi = []
    for p in pi:
        model += (xsum(u[i][j] for (i,j) in data.A <= K+1)
    
    # Contrainte 10
    for i in data.V:
        model += (t[i] <= L-1)

    # Contrainte 11
    for (i,j) in data.A:
            if(i != j):
                model += ( (t[i]-t[j]+(data.nb_pair+2)*y[i][j]) <= data.nb_pair+1)
    
    # Contrainte 12
    for i in data.N:
        model += (t[i] == 0)
     
    
    # Limitation du nombre de processeurs
    model.threads = 1  
    # Lancement du chronomètre
    start = time.perf_counter()
    # Résolution du modèle
    status = model.optimize(max_seconds=120)  # temps limite = 120s
    # Arrêt du chronomètre et calcul du temps de résolution
    runtime = time.perf_counter() - start
    
    print("\n----------------------------------")
    if status == OptimizationStatus.OPTIMAL:
        print("Status de la résolution: OPTIMAL")
    elif status == OptimizationStatus.FEASIBLE:
        print("Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE")
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print("Status de la résolution: TEMPS LIMITE et AUCUNE SOLUTION CALCULEE")
    elif status == OptimizationStatus.INFEASIBLE or status == OptimizationStatus.INT_INFEASIBLE:
        print("Status de la résolution: IRREALISABLE")
    elif status == OptimizationStatus.UNBOUNDED:
        print("Status de la résolution: NON BORNE")
        
    print("Temps de résolution (s) : ", runtime)    
    print("----------------------------------")

    # Si le modèle a été résolu à l'optimalité ou si une solution a été trouvée dans le temps limite accordé
    if model.num_solutions>0:
        print("Valeur de la fonction objectif de la solution calculée : ", round(model.objective_value,2))  # ne pas oublier d'arrondir si le coût doit être entier
   
    

SyntaxError: invalid syntax (<ipython-input-6-1978db264142>, line 46)

In [3]:
solve(Instancereader.Data(str(50),str(1)),5,5)

NameError: name 'solve' is not defined